# DATA Preprocessing

## DATA DIR

In [ ]:
import os

BASE_DIR = "/workspace/nas203/ds_RehabilitationMedicineData/data/body_key_point_Public_Data"
SAVE_DIR = "./data/Public_data"

# 라벨링 변경하기

## COCO

### Data Dir

In [ ]:
COCO_IMG_DIR = {s: os.path.join(BASE_DIR, "COCO", s) for s in ["train2017", "val2017", "test2017"]}
COCO_ANN_DIR = os.path.join(BASE_DIR, "COCO", "annotations")  # 주석 폴더
COCO_ANN_FILES  = {  # 주석 파일맵
    "train": os.path.join(COCO_ANN_DIR, "person_keypoints_train2017.json"),  # 학습 주석
    "val"  : os.path.join(COCO_ANN_DIR, "person_keypoints_val2017.json"),  # 검증 주석
}
COCO_SAVE_DIR = os.path.join(SAVE_DIR,"COCO") # COCO 이미지 및 라벨 저장 루트

### LANDMARK5 생성 및 저장

In [ ]:
import os, json, shutil  # 표준/입출력
from collections import defaultdict  # 딕트 기본값
from tqdm import tqdm  # 진행바

# ========== 변환 ==========  # 변환 루프
for split, ann_path in COCO_ANN_FILES.items():  # 분할별 순회
    print(f"\n[{split.upper()}] 변환 시작 ▶")  # 진행 출력
    with open(ann_path, "r") as f:  # 주석 열기
        coco = json.load(f)  # JSON 로드

    images = {img["id"]: img for img in coco["images"]}  # 이미지 인덱스
    labels_dict = defaultdict(list)  # 라벨 누적

    # --- 1) annotation 파싱 ---  # 파싱 섹션
    for ann in tqdm(coco["annotations"], desc=f"{split}: parse anns", total=len(coco["annotations"])):  # 주석 순회
        if ann["num_keypoints"] == 0:  # 키포인트 없음
            continue  # 건너뛰기

        img_id  = ann["image_id"]  # 이미지 ID
        imginfo = images[img_id]  # 이미지 정보
        W, H    = imginfo["width"], imginfo["height"]  # 폭과 높이
        if W == 0 or H == 0:  # 유효 크기 확인
            continue  # 건너뛰기

        bx, by, bw, bh = ann["bbox"]  # BBox 추출
        xc = (bx + bw / 2) / W  # cx 정규화
        yc = (by + bh / 2) / H  # cy 정규화
        bw /= W  # w 정규화
        bh /= H  # h 정규화

        kpts = ann["keypoints"]  # 키포인트 배열
        if len(kpts) < 51:  # 17점*3 검증
            continue  # 건너뛰기

        # --- neck 계산 ---  # 목점 계산
        LS, RS = 5, 6  # 어깨 인덱스
        x1, y1, v1 = kpts[LS*3], kpts[LS*3+1], kpts[LS*3+2]  # 좌측 어깨
        x2, y2, v2 = kpts[RS*3], kpts[RS*3+1], kpts[RS*3+2]  # 우측 어깨
        nx = (x1 + x2) / 2 / W  # 목 x 정규화
        ny = (y1 + y2) / 2 / H  # 목 y 정규화
        nv = float(min(v1, v2))  # 목 가시성

        reduced_kpts = [nx, ny, nv]  # 축약 키포인트 시작

        # --- LW(9), RW(10), LA(15), RA(16) ---  # 선택점 추가
        for idx in [9, 10, 15, 16]:  # 인덱스 순회
            x, y, v = kpts[idx * 3], kpts[idx * 3 + 1], kpts[idx * 3 + 2]  # 좌표/가시성
            reduced_kpts += [x / W, y / H, float(v)]  # 정규화 추가

        # 모든 keypoint가 가려졌으면 skip  # 전부 가림 시 스킵
        if all(v == 0 for v in reduced_kpts[2::3]):  # 가시성 검사
            continue  # 건너뛰기

        line = [0, xc, yc, bw, bh] + reduced_kpts  # 라벨 한 줄
        labels_dict[img_id].append(' '.join(f"{x:.6f}" for x in line))  # 포맷 저장

    # --- 2) 디렉터리 준비 ---  # 출력 경로 준비
    img_out_dir = os.path.join(COCO_SAVE_DIR, split, "images")  # 이미지 출력
    lbl_out_dir = os.path.join(COCO_SAVE_DIR, split, "labels5")  # 라벨 출력
    os.makedirs(img_out_dir, exist_ok=True)  # 폴더 생성
    os.makedirs(lbl_out_dir, exist_ok=True)  # 폴더 생성

    # --- 3) 이미지 & 라벨 저장 ---  # 저장 섹션
    for img_id, label_lines in tqdm(labels_dict.items(), desc=f"{split}: save files", total=len(labels_dict)):  # 파일 순회
        file_name = images[img_id]["file_name"]  # 파일명
        src_path  = os.path.join(COCO_IMG_DIR[f"{split}2017"], file_name)  # 원본 경로
        dst_img   = os.path.join(img_out_dir, file_name)  # 대상 이미지
        dst_lbl   = os.path.join(lbl_out_dir, os.path.splitext(file_name)[0] + ".txt")  # 대상 라벨

        if not os.path.exists(src_path):  # 원본 확인
            continue  # 건너뛰기
        shutil.copy2(src_path, dst_img)  # 이미지 복사

        with open(dst_lbl, "w") as f:  # 라벨 쓰기
            f.write("\n".join(label_lines) + "\n")  # 라벨 저장

    print(f"✅ {split} 완료: {len(labels_dict)} images / {sum(len(v) for v in labels_dict.values())} persons")  # 요약 출력


In [ ]:
import os
from pathlib import Path

# ---- 1) 특정 구조(COCO) 카운트 ----
SAVE_BASE = "./data/Public_data/COCO"

for split in ["train", "val"]:
    img_dir = os.path.join(SAVE_BASE, split, "images")
    lbl_dir = os.path.join(SAVE_BASE, split, "labels5")

    img_count = sum(1 for f in os.listdir(img_dir)
                    if f.lower().endswith((".jpg", ".jpeg", ".png")))
    lbl_count = sum(1 for f in os.listdir(lbl_dir) if f.lower().endswith(".txt"))

    # 이미지-라벨 매칭 상태도 함께 체크
    img_stems = {os.path.splitext(f)[0] for f in os.listdir(img_dir)
                 if f.lower().endswith((".jpg", ".jpeg", ".png"))}
    lbl_stems = {os.path.splitext(f)[0] for f in os.listdir(lbl_dir)
                 if f.lower().endswith(".txt")}
    missing_lbl = img_stems - lbl_stems
    missing_img = lbl_stems - img_stems

    print(f"[{split}] images: {img_count:,} | labels: {lbl_count:,}")
    print(f"    ├─ 라벨 없는 이미지: {len(missing_lbl):,}")
    print(f"    └─ 이미지 없는 라벨: {len(missing_img):,}")

# ---- 2) 범용: 임의 폴더의 파일 개수(확장자/재귀 옵션) ----
def count_files(dirpath, exts=None, recursive=True):
    """
    dirpath: 폴더 경로
    exts: ('jpg','png','txt') 처럼 확장자 튜플/리스트 (None이면 전체)
    recursive: 하위 폴더까지 포함 여부
    """
    dirpath = Path(dirpath)
    if exts:
        exts = tuple(x.lower().lstrip(".") for x in exts)

    n = 0
    it = dirpath.rglob("*") if recursive else dirpath.iterdir()
    for p in it:
        if p.is_file():
            if not exts:
                n += 1
            else:
                if p.suffix.lower().lstrip(".") in exts:
                    n += 1
    return n



[train] images: 55,424 | labels: 55,424
    ├─ 라벨 없는 이미지: 0
    └─ 이미지 없는 라벨: 0
[val] images: 2,293 | labels: 2,293
    ├─ 라벨 없는 이미지: 0
    └─ 이미지 없는 라벨: 0


In [ ]:
import os
import random
import cv2
import numpy as np
import matplotlib.pyplot as plt

plt.rcParams['figure.dpi'] = 180  # 선명도 업

base_dir  = "./data/Public_data/COCO"
split = "val"
img_dir = os.path.join(base_dir, split, "images")
label_dir = os.path.join(base_dir, split, "labels5")

all_labels = [f for f in os.listdir(label_dir) if f.endswith(".txt")]
sample_labels = random.sample(all_labels, min(5, len(all_labels)))

def find_image_path(root, stem):
    for ext in [".jpg", ".jpeg", ".png"]:
        p = os.path.join(root, stem + ext)
        if os.path.isfile(p):
            return p
    return None

def kpts_to_pixels_image_norm(kpts, w, h):
    pts = []
    for i in range(0, len(kpts), 3):
        x, y, v = kpts[i:i+3]
        pts.append((int(x * w), int(y * h), v))
    return pts

def kpts_to_pixels_box_norm(kpts, xc, yc, bw, bh, w, h):
    x0 = (xc - bw / 2.0) * w
    y0 = (yc - bh / 2.0) * h
    bw_px = bw * w
    bh_px = bh * h
    pts = []
    for i in range(0, len(kpts), 3):
        xr, yr, v = kpts[i:i+3]
        x = int(x0 + xr * bw_px)
        y = int(y0 + yr * bh_px)
        pts.append((x, y, v))
    return pts

def choose_projection(kpts, xc, yc, bw, bh, w, h):
    pts_img = kpts_to_pixels_image_norm(kpts, w, h)
    pts_box = kpts_to_pixels_box_norm(kpts, xc, yc, bw, bh, w, h)

    def in_bound_ratio(pts):
        if not pts: return 0.0
        cnt = sum(1 for x,y,v in pts if 0 <= x < w and 0 <= y < h and v > 0)
        return cnt / (len(pts) or 1)

    return pts_img if in_bound_ratio(pts_img) >= in_bound_ratio(pts_box) else pts_box

def draw_keypoints_with_ids(image, pts, color=(0, 255, 0)):
    for idx, (x, y, v) in enumerate(pts, start=1):
        if v <= 0:
            continue
        cv2.circle(image, (x, y), 8, color, -1, lineType=cv2.LINE_AA)  # 점 더 큼
        label = str(idx)
        (tw, th), _ = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 0.9, 2)  # 글씨 더 큼
        bg_tl = (x + 10, y - th - 8)
        bg_br = (x + 10 + tw + 8, y - 2)
        cv2.rectangle(image, bg_tl, bg_br, (0, 0, 0), thickness=-1)
        cv2.putText(image, label, (x + 14, y - 6),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 255, 255), 2, cv2.LINE_AA)
    return image

palette = [
    (0,255,0), (255,0,0), (0,200,255), (255,140,0), (200,0,200),
    (255,255,0), (0,255,255), (255,105,180)
]

for i, lbl_file in enumerate(sample_labels):
    stem = os.path.splitext(lbl_file)[0]
    img_path = find_image_path(img_dir, stem)
    if img_path is None:
        print(f"[경고] {stem} 이미지 없음")
        continue

    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    h, w = img.shape[:2]

    with open(os.path.join(label_dir, lbl_file), "r") as f:
        for li, line in enumerate(f):
            vals = list(map(float, line.strip().split()))
            if len(vals) < 5 + 3:
                continue
            cls, xc, yc, bw, bh = vals[:5]
            kpts = vals[5:]
            pts = choose_projection(kpts, xc, yc, bw, bh, w, h)
            color = palette[li % len(palette)]
            img = draw_keypoints_with_ids(img, pts, color=color)

    # ★ 한 장씩 크게 표시
    plt.figure(figsize=(4, 3))  # 더 크게 보고 싶으면 (20, 15) 등으로 조정
    plt.imshow(img)
    plt.title(os.path.basename(img_path), fontsize=16)
    plt.axis("off")
    plt.tight_layout()
    plt.show()


## DWPOSE

### data dir

In [ ]:
DW_IMG_DIR= os.path.join(BASE_DIR, "DWPOSE", "yolo_format")
DW_SAVE_DIR = os.path.join(SAVE_DIR,"DWPOSE") # COCO 이미지 및 라벨 저장 루트

### landmark5 생성 및 저장

In [ ]:
import os
import shutil
from tqdm import tqdm


# DWPOSE 기준 빨간 점 keypoint 인덱스 (목, 양 손목, 양 발목)
keypoint_idxs = [13, 4, 5, 10, 11]

IMG_EXTS = [".jpg", ".jpeg", ".png", ".JPG", ".JPEG", ".PNG"]

def find_image_path(img_root, stem):
    for ext in IMG_EXTS:
        p = os.path.join(img_root, stem + ext)
        if os.path.isfile(p):
            return p
    return None

for split in ["train", "val"]:
    print(f"▶ {split} 세트 변환 중...")

    original_lbl_dir   = os.path.join(DW_IMG_DIR, split, "labels")
    original_img_dir   = os.path.join(DW_IMG_DIR, split, "images")
    reduced_lbl_dir    = os.path.join(DW_SAVE_DIR, split, "labels5")
    reduced_image_dir  = os.path.join(DW_SAVE_DIR, split, "images")

    os.makedirs(reduced_lbl_dir, exist_ok=True)
    os.makedirs(reduced_image_dir, exist_ok=True)

    label_files = sorted([f for f in os.listdir(original_lbl_dir) if f.endswith(".txt")])
    file_count, line_count, img_count = 0, 0, 0

    # tqdm 적용
    for fname in tqdm(label_files, desc=f"{split} 변환 진행"):
        input_path  = os.path.join(original_lbl_dir, fname)
        output_path = os.path.join(reduced_lbl_dir, fname)

        lines_out = []
        with open(input_path, "r") as f:
            for line in f:
                vals = list(map(float, line.strip().split()))
                if len(vals) < 5 + 3 * (max(keypoint_idxs) + 1):
                    continue

                cls, xc, yc, bw, bh = vals[:5]
                kpts = vals[5:]

                # 모든 keypoint 가려진 경우 skip
                visibility = [kpts[idx * 3 + 2] for idx in keypoint_idxs]
                if all(v == 0 for v in visibility):
                    continue

                reduced_kpts = []
                for idx in keypoint_idxs:
                    reduced_kpts.extend(kpts[idx * 3 : idx * 3 + 3])  # [x, y, v]

                new_line = [cls, xc, yc, bw, bh] + reduced_kpts
                lines_out.append(' '.join(f"{v:.6f}" for v in new_line))

        # 변환된 라인 있으면 저장 + 이미지 복사
        if lines_out:
            # 1) 라벨 저장
            with open(output_path, 'w') as fw:
                fw.write('\n'.join(lines_out) + '\n')
            file_count += 1
            line_count += len(lines_out)

            # 2) 이미지 복사(확장자 자동 탐색)
            stem = os.path.splitext(fname)[0]
            src_img = find_image_path(original_img_dir, stem)
            if src_img is not None:
                dst_img = os.path.join(reduced_image_dir, os.path.basename(src_img))
                # 복사(메타데이터 보존). 용량 아끼려면 os.symlink 사용 가능(파일시스템/권한 허용 시)
                shutil.copy2(src_img, dst_img)
                img_count += 1
            else:
                # 이미지가 없을 수도 있으니 경고만 표시
                print(f"[경고] 이미지 없음: {stem} (labels OK)")

    print(f"✅ 저장 완료: {reduced_lbl_dir} (라벨 파일 {file_count}개, 라인 {line_count}개)")
    print(f"✅ 이미지 복사 완료: {reduced_image_dir} (이미지 {img_count}개)")

print("\n🎉 전체 라벨5 + 이미지 복사 완료!")


In [ ]:
import os
from pathlib import Path

# ---- 1) 특정 구조(COCO) 카운트 ----
SAVE_BASE = "./data/Public_data/DWPOSE"

for split in ["train", "val"]:
    img_dir = os.path.join(SAVE_BASE, split, "images")
    lbl_dir = os.path.join(SAVE_BASE, split, "labels5")

    img_count = sum(1 for f in os.listdir(img_dir)
                    if f.lower().endswith((".jpg", ".jpeg", ".png")))
    lbl_count = sum(1 for f in os.listdir(lbl_dir) if f.lower().endswith(".txt"))

    # 이미지-라벨 매칭 상태도 함께 체크
    img_stems = {os.path.splitext(f)[0] for f in os.listdir(img_dir)
                 if f.lower().endswith((".jpg", ".jpeg", ".png"))}
    lbl_stems = {os.path.splitext(f)[0] for f in os.listdir(lbl_dir)
                 if f.lower().endswith(".txt")}
    missing_lbl = img_stems - lbl_stems
    missing_img = lbl_stems - img_stems

    print(f"[{split}] images: {img_count:,} | labels: {lbl_count:,}")
    print(f"    ├─ 라벨 없는 이미지: {len(missing_lbl):,}")
    print(f"    └─ 이미지 없는 라벨: {len(missing_img):,}")

# ---- 2) 범용: 임의 폴더의 파일 개수(확장자/재귀 옵션) ----
def count_files(dirpath, exts=None, recursive=True):
    """
    dirpath: 폴더 경로
    exts: ('jpg','png','txt') 처럼 확장자 튜플/리스트 (None이면 전체)
    recursive: 하위 폴더까지 포함 여부
    """
    dirpath = Path(dirpath)
    if exts:
        exts = tuple(x.lower().lstrip(".") for x in exts)

    n = 0
    it = dirpath.rglob("*") if recursive else dirpath.iterdir()
    for p in it:
        if p.is_file():
            if not exts:
                n += 1
            else:
                if p.suffix.lower().lstrip(".") in exts:
                    n += 1
    return n

# # 사용 예시(원하면 주석 해제해서 실행):
# print(count_files("/some/path"))                         # 전체 파일(재귀)
# print(count_files("/some/path", exts=["jpg","png"]))     # 이미지 개수
# print(count_files("/some/path", exts=["txt"], recursive=False))  # 현재 폴더 txt 개수


In [ ]:
import os
import random
import cv2
import numpy as np
import matplotlib.pyplot as plt

plt.rcParams['figure.dpi'] = 180  # 선명도 업

split = "val"
img_dir = os.path.join(DW_SAVE_DIR, split, "images")
label_dir = os.path.join(DW_SAVE_DIR, split, "labels5")

all_labels = [f for f in os.listdir(label_dir) if f.endswith(".txt")]
sample_labels = random.sample(all_labels, min(5, len(all_labels)))

def find_image_path(root, stem):
    for ext in [".jpg", ".jpeg", ".png"]:
        p = os.path.join(root, stem + ext)
        if os.path.isfile(p):
            return p
    return None

def kpts_to_pixels_image_norm(kpts, w, h):
    pts = []
    for i in range(0, len(kpts), 3):
        x, y, v = kpts[i:i+3]
        pts.append((int(x * w), int(y * h), v))
    return pts

def kpts_to_pixels_box_norm(kpts, xc, yc, bw, bh, w, h):
    x0 = (xc - bw / 2.0) * w
    y0 = (yc - bh / 2.0) * h
    bw_px = bw * w
    bh_px = bh * h
    pts = []
    for i in range(0, len(kpts), 3):
        xr, yr, v = kpts[i:i+3]
        x = int(x0 + xr * bw_px)
        y = int(y0 + yr * bh_px)
        pts.append((x, y, v))
    return pts

def choose_projection(kpts, xc, yc, bw, bh, w, h):
    pts_img = kpts_to_pixels_image_norm(kpts, w, h)
    pts_box = kpts_to_pixels_box_norm(kpts, xc, yc, bw, bh, w, h)

    def in_bound_ratio(pts):
        if not pts: return 0.0
        cnt = sum(1 for x,y,v in pts if 0 <= x < w and 0 <= y < h and v > 0)
        return cnt / (len(pts) or 1)

    return pts_img if in_bound_ratio(pts_img) >= in_bound_ratio(pts_box) else pts_box

def draw_keypoints_with_ids(image, pts, color=(0, 255, 0)):
    for idx, (x, y, v) in enumerate(pts, start=1):
        if v <= 0:
            continue
        cv2.circle(image, (x, y), 8, color, -1, lineType=cv2.LINE_AA)  # 점 더 큼
        label = str(idx)
        (tw, th), _ = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 0.9, 2)  # 글씨 더 큼
        bg_tl = (x + 10, y - th - 8)
        bg_br = (x + 10 + tw + 8, y - 2)
        cv2.rectangle(image, bg_tl, bg_br, (0, 0, 0), thickness=-1)
        cv2.putText(image, label, (x + 14, y - 6),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 255, 255), 2, cv2.LINE_AA)
    return image

palette = [
    (0,255,0), (255,0,0), (0,200,255), (255,140,0), (200,0,200),
    (255,255,0), (0,255,255), (255,105,180)
]

for i, lbl_file in enumerate(sample_labels):
    stem = os.path.splitext(lbl_file)[0]
    img_path = find_image_path(img_dir, stem)
    if img_path is None:
        print(f"[경고] {stem} 이미지 없음")
        continue

    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    h, w = img.shape[:2]

    with open(os.path.join(label_dir, lbl_file), "r") as f:
        for li, line in enumerate(f):
            vals = list(map(float, line.strip().split()))
            if len(vals) < 5 + 3:
                continue
            cls, xc, yc, bw, bh = vals[:5]
            kpts = vals[5:]
            pts = choose_projection(kpts, xc, yc, bw, bh, w, h)
            color = palette[li % len(palette)]
            img = draw_keypoints_with_ids(img, pts, color=color)

    # ★ 한 장씩 크게 표시
    plt.figure(figsize=(4, 3))  # 더 크게 보고 싶으면 (20, 15) 등으로 조정
    plt.imshow(img)
    plt.title(os.path.basename(img_path), fontsize=16)
    plt.axis("off")
    plt.tight_layout()
    plt.show()


## MPII

### DATA DIR

In [ ]:
MP_SAVE_DIR = os.path.join(SAVE_DIR,"MPII") # COCO 이미지 및 라벨 저장 루트



In [ ]:
import os  # OS 유틸
import shutil  # 파일 복사
import scipy.io as sio  # MAT 로드
import numpy as np  # 수치 연산
from PIL import Image  # 이미지 I/O
from tqdm import tqdm  # 진행바
from collections import defaultdict  # 기본 딕트

# ============== 설정 ==============  # 설정 섹션
SEED = 42  # 랜덤 시드
MAT_FILE   = '/workspace/nas203/ds_RehabilitationMedicineData/data/body_key_point_Public_Data/MPII/mpii_human_pose_v1_u12_2/mpii_human_pose_v1_u12_1.mat'  # 주석 경로
SRC_IMAGES = '/workspace/nas203/ds_RehabilitationMedicineData/data/body_key_point_Public_Data/MPII/mpii/images'  # 이미지 폴더

# 5점(목, 왼손목, 오른손목, 왼발목, 오른발목)으로 축약: 사용자 지정 인덱스  # 선택 키포인트
SELECTED_IDS = [8, 15, 10, 5, 0]  # 선택 인덱스

# 출력 디렉토리 구성  # 출력 경로
DIRS = {  # 분할별 경로
    'train': {  # 학습 폴더
        'images': os.path.join(MP_SAVE_DIR, 'train', 'images'),  # 학습 이미지
        'labels5': os.path.join(MP_SAVE_DIR, 'train', 'labels5')  # 학습 라벨
    },
    'val': {  # 검증 폴더
        'images': os.path.join(MP_SAVE_DIR, 'val', 'images'),  # 검증 이미지
        'labels5': os.path.join(MP_SAVE_DIR, 'val', 'labels5')  # 검증 라벨
    }
}
for split in DIRS:  # 분할 순회
    os.makedirs(DIRS[split]['images'], exist_ok=True)  # 이미지 폴더 생성
    os.makedirs(DIRS[split]['labels5'], exist_ok=True)  # 라벨 폴더 생성

# ============== .mat 로드 ==============  # MAT 읽기
data     = sio.loadmat(MAT_FILE, squeeze_me=True, struct_as_record=False)  # MAT 로드
release  = data['RELEASE']  # 메타 추출
annolist = release.annolist  # 어노 리스트
is_train = np.asarray(release.img_train).astype(bool)  # 학습 플래그

# ============== train만 추출 → 이미지 기준 9:1 분할 ==============  # 분할 설정
# annolist는 이미지 단위 항목. 같은 이미지가 중복될 가능성 방지 위해 이미지명으로 분할 리스트 생성  # 설명 주석
train_indices = [i for i in range(len(annolist)) if is_train[i]]  # 학습 인덱스

# 고유 이미지명 기반으로 분할  # 고유화 처리
img_to_idx = defaultdict(list)  # 이미지→인덱스
for i in train_indices:  # 인덱스 순회
    img_name = os.path.basename(annolist[i].image.name)  # 파일명 추출
    img_to_idx[img_name].append(i)  # 매핑 추가

unique_imgs = list(img_to_idx.keys())  # 고유 목록
rng = np.random.default_rng(SEED)  # 난수기 초기화
rng.shuffle(unique_imgs)  # 섞기

cut = int(len(unique_imgs) * 0.9)  # 9:1 분할점
train_imgs = set(unique_imgs[:cut])  # 학습 집합
val_imgs   = set(unique_imgs[cut:])  # 검증 집합

def which_split(img_name):  # 분할 판단
    return 'train' if img_name in train_imgs else 'val'  # 분기 반환

# ============== 변환 함수들 ==============  # 함수 섹션
def _to_list(x):  # 리스트 변환
    """numpy scalar/array 또는 단일 객체를 리스트로 평탄화"""  # 함수 설명
    if x is None:  # 없음 처리
        return []  # 빈 리스트
    if isinstance(x, np.ndarray):  # 배열 여부
        return x.tolist()  # 리스트화
    return [x]  # 단일 감싸기

def build_labels_for_ann(ann, w, h):  # 라벨 생성
    """
    ann(한 이미지 항목)에서 사람별 라인들을 생성.
    반환: ['cls xc yc bw bh kpts...', ...]
    """  # 동작 설명
    rects = _to_list(ann.annorect)  # 사람 박스들
    lines = []  # 결과 라인들

    for r in rects:  # 각 박스 순회
        if r is None or not hasattr(r, 'annopoints'):  # 포인트 확인
            continue  # 스킵

        aps_list = _to_list(r.annopoints)  # 포인트 세트
        all_pts = []  # 포인트 누적
        for ap in aps_list:  # 세트 순회
            pts = getattr(ap, 'point', None)  # 포인트 취득
            all_pts += _to_list(pts)  # 리스트화 합침

        if not all_pts:  # 포인트 없음
            continue  # 스킵

        # id 최대값 기준으로 테이블 만들기  # 테이블 크기
        try:
            max_id = max(int(p.id) for p in all_pts)  # 최대 id
        except Exception:  # 실패 처리
            continue  # 스킵

        # [x, y, v] 테이블 구성 (v는 score 사용; 없으면 1.0)  # 테이블 생성
        ktab = [[0.0, 0.0, 0.0] for _ in range(max_id + 1)]  # 초기화
        xs, ys = [], []  # 좌표 누적
        for p in all_pts:  # 포인트 순회
            try:
                pid = int(p.id)  # 포인트 id
                x = float(p.x); y = float(p.y)  # 좌표 추출
                v = float(getattr(p, 'score', 1.0))  # 가시성
            except Exception:  # 파싱 실패
                continue  # 스킵
            ktab[pid] = [x, y, v]  # 표 채움
            xs.append(x); ys.append(y)  # 좌표 누적

        if not xs or not ys:  # 좌표 없음
            continue  # 스킵

        # bbox (정규화)  # 박스 계산
        x_min, x_max = min(xs), max(xs)  # x 범위
        y_min, y_max = min(ys), max(ys)  # y 범위
        bw = (x_max - x_min) / w  # 폭 정규화
        bh = (y_max - y_min) / h  # 높이 정규화
        if bw <= 0 or bh <= 0:  # 비정상 크기
            continue  # 스킵

        xc = ((x_min + x_max) / 2) / w  # 중심 x
        yc = ((y_min + y_max) / 2) / h  # 중심 y

        # 5점만 정규화하여 추출  # 선택점 처리
        flat = []  # 키점 버퍼
        for idx in SELECTED_IDS:  # 선택 순회
            if 0 <= idx < len(ktab):  # 범위 확인
                x, y, v = ktab[idx]  # 값 취득
            else:
                x, y, v = 0.0, 0.0, 0.0  # 기본값
            flat += [x / w, y / h, v]  # 정규화 추가

        vals = [0, xc, yc, bw, bh] + flat  # 한 줄 구성
        lines.append(' '.join(f"{v:.6f}" for v in vals))  # 문자열화

    return lines  # 라벨 반환

# ============== 본변환: 이미지 복사 + labels5 작성 ==============  # 본 처리
stats = {  # 통계 딕트
    'train': {'files': 0, 'lines': 0, 'copied': 0, 'no_src': 0, 'no_pts': 0},  # 학습 통계
    'val':   {'files': 0, 'lines': 0, 'copied': 0, 'no_src': 0, 'no_pts': 0},  # 검증 통계
}

for i in tqdm(train_indices, desc="MPII train → (train/val 9:1) + labels5", unit="img", dynamic_ncols=True):  # 인덱스 루프
    ann = annolist[i]  # 주석 항목
    img_name = os.path.basename(ann.image.name)  # 파일명
    split = which_split(img_name)  # 분할 결정

    src_img = os.path.join(SRC_IMAGES, img_name)  # 원본 경로
    if not os.path.isfile(src_img):  # 존재 확인
        stats[split]['no_src'] += 1  # 원본 없음 카운트
        continue  # 스킵

    # 이미지 크기  # 크기 읽기
    try:
        with Image.open(src_img) as im:  # 이미지 열기
            w, h = im.size  # 폭 높이
    except Exception:  # 실패 시
        stats[split]['no_src'] += 1  # 원본 오류 카운트
        continue  # 스킵

    # 라벨 생성  # 키점 생성
    lines = build_labels_for_ann(ann, w, h)  # 라벨 빌드
    if not lines:  # 비어있음
        stats[split]['no_pts'] += 1  # 포인트 없음
        continue  # 스킵

    # 이미지 복사(한 번만)  # 이미지 복사
    dst_img = os.path.join(DIRS[split]['images'], img_name)  # 대상 경로
    if not os.path.isfile(dst_img):  # 미존재시
        os.makedirs(os.path.dirname(dst_img), exist_ok=True)  # 폴더 생성
        try:
            shutil.copy2(src_img, dst_img)  # 이미지 복사
            stats[split]['copied'] += 1  # 복사 카운트
        except Exception:
            # 복사 실패해도 라벨은 쓸 수 있게 진행  # 실패 허용
            pass  # 계속 진행

    # 라벨 저장 (사람 수만큼 라인)  # 라벨 저장
    dst_lbl = os.path.join(DIRS[split]['labels5'], os.path.splitext(img_name)[0] + '.txt')  # 라벨 경로
    with open(dst_lbl, 'w') as f:  # 파일 열기
        f.write('\n'.join(lines))  # 내용 기록

    stats[split]['files'] += 1  # 파일 수 증가
    stats[split]['lines'] += len(lines)  # 라인 수 증가

# ============== 요약 출력 ==============  # 결과 요약
print("\n✅ 완료: MPII train → (train/val 9:1) 분할 + YOLO labels5 생성")  # 완료 출력
for split in ('train', 'val'):  # 분할 순회
    s = stats[split]  # 통계 참조
    print(f"[{split}]")  # 분할 출력
    print(f"  ├─ 라벨 파일 수      : {s['files']}")  # 라벨 수
    print(f"  ├─ 총 라인(사람 수)   : {s['lines']}")  # 총 라인
    print(f"  ├─ 복사된 이미지 수   : {s['copied']}")  # 복사 수
    print(f"  ├─ 스킵(원본 이미지X) : {s['no_src']}")  # 원본 없음
    print(f"  └─ 스킵(포인트 없음)  : {s['no_pts']}")  # 포인트 없음


In [ ]:
import os
from pathlib import Path

# ---- 1) 특정 구조(COCO) 카운트 ----
SAVE_BASE = "./data/Public_data/MPII"

for split in ["train", "val"]:
    img_dir = os.path.join(SAVE_BASE, split, "images")
    lbl_dir = os.path.join(SAVE_BASE, split, "labels5")

    img_count = sum(1 for f in os.listdir(img_dir)
                    if f.lower().endswith((".jpg", ".jpeg", ".png")))
    lbl_count = sum(1 for f in os.listdir(lbl_dir) if f.lower().endswith(".txt"))

    # 이미지-라벨 매칭 상태도 함께 체크
    img_stems = {os.path.splitext(f)[0] for f in os.listdir(img_dir)
                 if f.lower().endswith((".jpg", ".jpeg", ".png"))}
    lbl_stems = {os.path.splitext(f)[0] for f in os.listdir(lbl_dir)
                 if f.lower().endswith(".txt")}
    missing_lbl = img_stems - lbl_stems
    missing_img = lbl_stems - img_stems

    print(f"[{split}] images: {img_count:,} | labels: {lbl_count:,}")
    print(f"    ├─ 라벨 없는 이미지: {len(missing_lbl):,}")
    print(f"    └─ 이미지 없는 라벨: {len(missing_img):,}")

# ---- 2) 범용: 임의 폴더의 파일 개수(확장자/재귀 옵션) ----
def count_files(dirpath, exts=None, recursive=True):
    """
    dirpath: 폴더 경로
    exts: ('jpg','png','txt') 처럼 확장자 튜플/리스트 (None이면 전체)
    recursive: 하위 폴더까지 포함 여부
    """
    dirpath = Path(dirpath)
    if exts:
        exts = tuple(x.lower().lstrip(".") for x in exts)

    n = 0
    it = dirpath.rglob("*") if recursive else dirpath.iterdir()
    for p in it:
        if p.is_file():
            if not exts:
                n += 1
            else:
                if p.suffix.lower().lstrip(".") in exts:
                    n += 1
    return n


In [ ]:
import os
import random
import cv2
import numpy as np
import matplotlib.pyplot as plt

plt.rcParams['figure.dpi'] = 180  # 선명도 업


split = "val"
img_dir = os.path.join(MP_SAVE_DIR, split, "images")
label_dir = os.path.join(MP_SAVE_DIR, split, "labels5")

all_labels = [f for f in os.listdir(label_dir) if f.endswith(".txt")]
sample_labels = random.sample(all_labels, min(5, len(all_labels)))

def find_image_path(root, stem):
    for ext in [".jpg", ".jpeg", ".png"]:
        p = os.path.join(root, stem + ext)
        if os.path.isfile(p):
            return p
    return None

def kpts_to_pixels_image_norm(kpts, w, h):
    pts = []
    for i in range(0, len(kpts), 3):
        x, y, v = kpts[i:i+3]
        pts.append((int(x * w), int(y * h), v))
    return pts

def kpts_to_pixels_box_norm(kpts, xc, yc, bw, bh, w, h):
    x0 = (xc - bw / 2.0) * w
    y0 = (yc - bh / 2.0) * h
    bw_px = bw * w
    bh_px = bh * h
    pts = []
    for i in range(0, len(kpts), 3):
        xr, yr, v = kpts[i:i+3]
        x = int(x0 + xr * bw_px)
        y = int(y0 + yr * bh_px)
        pts.append((x, y, v))
    return pts

def choose_projection(kpts, xc, yc, bw, bh, w, h):
    pts_img = kpts_to_pixels_image_norm(kpts, w, h)
    pts_box = kpts_to_pixels_box_norm(kpts, xc, yc, bw, bh, w, h)

    def in_bound_ratio(pts):
        if not pts: return 0.0
        cnt = sum(1 for x,y,v in pts if 0 <= x < w and 0 <= y < h and v > 0)
        return cnt / (len(pts) or 1)

    return pts_img if in_bound_ratio(pts_img) >= in_bound_ratio(pts_box) else pts_box

def draw_keypoints_with_ids(image, pts, color=(0, 255, 0)):
    for idx, (x, y, v) in enumerate(pts, start=1):
        if v <= 0:
            continue
        cv2.circle(image, (x, y), 8, color, -1, lineType=cv2.LINE_AA)  # 점 더 큼
        label = str(idx)
        (tw, th), _ = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 0.9, 2)  # 글씨 더 큼
        bg_tl = (x + 10, y - th - 8)
        bg_br = (x + 10 + tw + 8, y - 2)
        cv2.rectangle(image, bg_tl, bg_br, (0, 0, 0), thickness=-1)
        cv2.putText(image, label, (x + 14, y - 6),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 255, 255), 2, cv2.LINE_AA)
    return image

palette = [
    (0,255,0), (255,0,0), (0,200,255), (255,140,0), (200,0,200),
    (255,255,0), (0,255,255), (255,105,180)
]

for i, lbl_file in enumerate(sample_labels):
    stem = os.path.splitext(lbl_file)[0]
    img_path = find_image_path(img_dir, stem)
    if img_path is None:
        print(f"[경고] {stem} 이미지 없음")
        continue

    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    h, w = img.shape[:2]

    with open(os.path.join(label_dir, lbl_file), "r") as f:
        for li, line in enumerate(f):
            vals = list(map(float, line.strip().split()))
            if len(vals) < 5 + 3:
                continue
            cls, xc, yc, bw, bh = vals[:5]
            kpts = vals[5:]
            pts = choose_projection(kpts, xc, yc, bw, bh, w, h)
            color = palette[li % len(palette)]
            img = draw_keypoints_with_ids(img, pts, color=color)

    # ★ 한 장씩 크게 표시
    plt.figure(figsize=(4, 3))  # 더 크게 보고 싶으면 (20, 15) 등으로 조정
    plt.imshow(img)
    plt.title(os.path.basename(img_path), fontsize=16)
    plt.axis("off")
    plt.tight_layout()
    plt.show()


## AMC DATA

In [ ]:
# Jupyter 셀에서 실행
!apt-get update -y
!apt-get install -y fonts-nanum fonts-noto-cjk


In [ ]:
from pathlib import Path
import random, cv2
import matplotlib.pyplot as plt

# ==== 경로 ====
SRC_IMG_DIR   = Path("/workspace/nas203/ds_RehabilitationMedicineData/data/yolo_data/images")
SRC_LABEL_DIR = Path("/workspace/nas203/ds_RehabilitationMedicineData/data/yolo_data/labels")

# ==== 유틸 ====
def find_image_path(stem, exts=(".jpg",".jpeg",".png",".bmp")):
    for e in exts:
        p = SRC_IMG_DIR / f"{stem}{e}"
        if p.is_file():
            return str(p)
    return None

def draw_yolo_pose(img, line):
    parts = list(map(float, line.strip().split()))
    if len(parts) < 5: 
        return img
    cls = int(parts[0])
    xc, yc, bw, bh = parts[1:5]
    kpts = parts[5:]

    h, w = img.shape[:2]
    x1 = int((xc - bw/2) * w)
    y1 = int((yc - bh/2) * h)
    x2 = int((xc + bw/2) * w)
    y2 = int((yc + bh/2) * h)
    cv2.rectangle(img, (x1,y1), (x2,y2), (255,0,0), 2)
    cv2.putText(img, f"cls:{cls}", (x1, max(0,y1-5)),
                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,0,0), 1, cv2.LINE_AA)

    # keypoints (x,y,v) 반복
    for i in range(0, len(kpts), 3):
        x, y, v = kpts[i:i+3]
        if v > 0:
            px, py = int(x * w), int(y * h)
            cv2.circle(img, (px, py), 4, (0,255,0), -1, lineType=cv2.LINE_AA)
            cv2.putText(img, str(i//3+1), (px+5, py-5),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.45, (255,255,255), 1, cv2.LINE_AA)
    return img

# ==== 랜덤 N개 시각화 ====
N = 6
label_files = sorted(SRC_LABEL_DIR.glob("*.txt"))
samples = random.sample(label_files, min(N, len(label_files)))

cols = min(3, len(samples))
rows = (len(samples) + cols - 1) // cols
plt.figure(figsize=(5*cols, 5*rows))

for idx, lbl in enumerate(samples, 1):
    stem = lbl.stem
    img_path = find_image_path(stem)
    if img_path is None:
        continue
    img = cv2.cvtColor(cv2.imread(img_path), cv2.COLOR_BGR2RGB)

    with open(lbl, "r") as f:
        for line in f:
            img = draw_yolo_pose(img, line)

    plt.subplot(rows, cols, idx)
    plt.imshow(img); plt.axis("off"); plt.title(stem)

plt.tight_layout(); plt.show()


In [ ]:
import os
import shutil
import random
import numpy as np
from pathlib import Path
from tqdm import tqdm
from collections import defaultdict

# ================================
# 경로 설정
# ================================
ROOT = Path("/workspace/nas203/ds_RehabilitationMedicineData/data/yolo_data")
SRC_LABEL_DIR = ROOT / "labels"
SRC_IMAGE_DIR = ROOT / "images"  # 이미지 경로 (필요 시 수정)

DST_ROOT = Path("./data/Patient_data")
TRAIN_LABEL_DIR = DST_ROOT / "train" / "labels5"
VAL_LABEL_DIR = DST_ROOT / "val" / "labels5"
TRAIN_IMAGE_DIR = DST_ROOT / "train" / "images"
VAL_IMAGE_DIR = DST_ROOT / "val" / "images"

for d in [TRAIN_LABEL_DIR, VAL_LABEL_DIR, TRAIN_IMAGE_DIR, VAL_IMAGE_DIR]:
    d.mkdir(parents=True, exist_ok=True)

# ================================
# COCO 17 keypoints → 선택 규칙
# ================================
ID_LSH, ID_RSH = 5, 6
ID_LW, ID_RW   = 9, 10
ID_LA, ID_RA   = 15, 16

def build_line(cls, bbox, kps, h, w):
    """ YOLO txt 한 줄 변환 """
    nx  = (kps[ID_LSH,0] + kps[ID_RSH,0]) / 2 / w
    ny  = (kps[ID_LSH,1] + kps[ID_RSH,1]) / 2 / h
    nv  = int(min(kps[ID_LSH,2], kps[ID_RSH,2]))

    pts = [
        (nx, ny, nv),
        (kps[ID_LW,0]/w, kps[ID_LW,1]/h, int(kps[ID_LW,2])),
        (kps[ID_RW,0]/w, kps[ID_RW,1]/h, int(kps[ID_RW,2])),
        (kps[ID_LA,0]/w, kps[ID_LA,1]/h, int(kps[ID_LA,2])),
        (kps[ID_RA,0]/w, kps[ID_RA,1]/h, int(kps[ID_RA,2])),
    ]

    flat = []
    for x, y, v in pts:
        flat += [f"{x:.6f}", f"{y:.6f}", str(v)]

    return " ".join([cls] + bbox + flat)

# ================================
# 라벨 변환 (labels → labels5)
# ================================
print("🔄 라벨 변환 중...")
converted_labels = []
txt_files = sorted(SRC_LABEL_DIR.glob("*.txt"))

for txt_path in tqdm(txt_files, desc="라벨 파일 변환", unit="file"):
    out_lines = []
    with open(txt_path) as f:
        for ln in f:
            parts = ln.strip().split()
            if len(parts) < 5:
                continue
            cls, bbox, kplist = parts[0], parts[1:5], parts[5:]
            kps = np.array(kplist, float).reshape(-1,3)
            out_lines.append(build_line(cls, bbox, kps, h=1, w=1))

    if out_lines:
        converted_labels.append(txt_path.name)

print(f"✅ {len(converted_labels)}개 라벨 변환 완료")

# ================================
# 환자 단위 split (9:1)
# ================================
print("🔄 Train/Val split 중...")

# 환자 단위 그룹핑
patient_groups = defaultdict(list)
for fname in converted_labels:
    prefix = fname.split("_")[0]   # 언더바 앞 부분을 환자 ID로 사용
    patient_groups[prefix].append(fname)

patients = list(patient_groups.keys())
random.seed(42)
random.shuffle(patients)

val_ratio = 0.1
split_idx = int(len(patients) * (1 - val_ratio))
train_patients = patients[:split_idx]
val_patients = patients[split_idx:]

print(f"총 환자 수: {len(patients)} → Train: {len(train_patients)}, Val: {len(val_patients)}")

# ================================
# 파일 복사 (라벨+이미지)
# ================================
def copy_files(patients, label_dst, image_dst):
    for p in tqdm(patients, desc=f"{label_dst.parent.name} 환자 처리", unit="patient"):
        for fname in patient_groups[p]:
            # 라벨 파일 읽어서 변환 저장
            src_label = SRC_LABEL_DIR / fname
            dst_label = label_dst / fname
            out_lines = []
            with open(src_label) as f:
                for ln in f:
                    parts = ln.strip().split()
                    if len(parts) < 5:
                        continue
                    cls, bbox, kplist = parts[0], parts[1:5], parts[5:]
                    kps = np.array(kplist, float).reshape(-1,3)
                    out_lines.append(build_line(cls, bbox, kps, h=1, w=1))
            if out_lines:
                dst_label.write_text("\n".join(out_lines))

            # 이미지 복사
            stem = Path(fname).stem
            for ext in [".jpg", ".png", ".jpeg"]:
                src_img = SRC_IMAGE_DIR / f"{stem}{ext}"
                if src_img.exists():
                    shutil.copy(src_img, image_dst / src_img.name)
                    break

copy_files(train_patients, TRAIN_LABEL_DIR, TRAIN_IMAGE_DIR)
copy_files(val_patients, VAL_LABEL_DIR, VAL_IMAGE_DIR)

print("✅ 데이터 분할 및 저장 완료")




In [ ]:
import os
from pathlib import Path

# ---- 1) 특정 구조(COCO) 카운트 ----
SAVE_BASE = "./data/Patient_data"

for split in ["train", "val"]:
    img_dir = os.path.join(SAVE_BASE, split, "images")
    lbl_dir = os.path.join(SAVE_BASE, split, "labels5")

    img_count = sum(1 for f in os.listdir(img_dir)
                    if f.lower().endswith((".jpg", ".jpeg", ".png")))
    lbl_count = sum(1 for f in os.listdir(lbl_dir) if f.lower().endswith(".txt"))

    # 이미지-라벨 매칭 상태도 함께 체크
    img_stems = {os.path.splitext(f)[0] for f in os.listdir(img_dir)
                 if f.lower().endswith((".jpg", ".jpeg", ".png"))}
    lbl_stems = {os.path.splitext(f)[0] for f in os.listdir(lbl_dir)
                 if f.lower().endswith(".txt")}
    missing_lbl = img_stems - lbl_stems
    missing_img = lbl_stems - img_stems

    print(f"[{split}] images: {img_count:,} | labels: {lbl_count:,}")
    print(f"    ├─ 라벨 없는 이미지: {len(missing_lbl):,}")
    print(f"    └─ 이미지 없는 라벨: {len(missing_img):,}")

# ---- 2) 범용: 임의 폴더의 파일 개수(확장자/재귀 옵션) ----
def count_files(dirpath, exts=None, recursive=True):
    """
    dirpath: 폴더 경로
    exts: ('jpg','png','txt') 처럼 확장자 튜플/리스트 (None이면 전체)
    recursive: 하위 폴더까지 포함 여부
    """
    dirpath = Path(dirpath)
    if exts:
        exts = tuple(x.lower().lstrip(".") for x in exts)

    n = 0
    it = dirpath.rglob("*") if recursive else dirpath.iterdir()
    for p in it:
        if p.is_file():
            if not exts:
                n += 1
            else:
                if p.suffix.lower().lstrip(".") in exts:
                    n += 1
    return n



In [ ]:
import os
import random
import cv2
import numpy as np
import matplotlib.pyplot as plt

plt.rcParams['figure.dpi'] = 180  # 선명도 업

base_dir  = "./data/Patient_data"
split = "val"
img_dir =os.path.join(base_dir, split, "images")
label_dir = os.path.join(base_dir, split, "labels5")

all_labels = [f for f in os.listdir(label_dir) if f.endswith(".txt")]
sample_labels = random.sample(all_labels, min(5, len(all_labels)))

def find_image_path(root, stem):
    for ext in [".jpg", ".jpeg", ".png"]:
        p = os.path.join(root, stem + ext)
        if os.path.isfile(p):
            return p
    return None

def kpts_to_pixels_image_norm(kpts, w, h):
    pts = []
    for i in range(0, len(kpts), 3):
        x, y, v = kpts[i:i+3]
        pts.append((int(x * w), int(y * h), v))
    return pts

def kpts_to_pixels_box_norm(kpts, xc, yc, bw, bh, w, h):
    x0 = (xc - bw / 2.0) * w
    y0 = (yc - bh / 2.0) * h
    bw_px = bw * w
    bh_px = bh * h
    pts = []
    for i in range(0, len(kpts), 3):
        xr, yr, v = kpts[i:i+3]
        x = int(x0 + xr * bw_px)
        y = int(y0 + yr * bh_px)
        pts.append((x, y, v))
    return pts

def choose_projection(kpts, xc, yc, bw, bh, w, h):
    pts_img = kpts_to_pixels_image_norm(kpts, w, h)
    pts_box = kpts_to_pixels_box_norm(kpts, xc, yc, bw, bh, w, h)

    def in_bound_ratio(pts):
        if not pts: return 0.0
        cnt = sum(1 for x,y,v in pts if 0 <= x < w and 0 <= y < h and v > 0)
        return cnt / (len(pts) or 1)

    return pts_img if in_bound_ratio(pts_img) >= in_bound_ratio(pts_box) else pts_box

def draw_keypoints_with_ids(image, pts, color=(0, 255, 0)):
    for idx, (x, y, v) in enumerate(pts, start=1):
        if v <= 0:
            continue
        cv2.circle(image, (x, y), 8, color, -1, lineType=cv2.LINE_AA)  # 점 더 큼
        label = str(idx)
        (tw, th), _ = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 0.9, 2)  # 글씨 더 큼
        bg_tl = (x + 10, y - th - 8)
        bg_br = (x + 10 + tw + 8, y - 2)
        cv2.rectangle(image, bg_tl, bg_br, (0, 0, 0), thickness=-1)
        cv2.putText(image, label, (x + 14, y - 6),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 255, 255), 2, cv2.LINE_AA)
    return image

palette = [
    (0,255,0), (255,0,0), (0,200,255), (255,140,0), (200,0,200),
    (255,255,0), (0,255,255), (255,105,180)
]

for i, lbl_file in enumerate(sample_labels):
    stem = os.path.splitext(lbl_file)[0]
    img_path = find_image_path(img_dir, stem)
    if img_path is None:
        print(f"[경고] {stem} 이미지 없음")
        continue

    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    h, w = img.shape[:2]

    with open(os.path.join(label_dir, lbl_file), "r") as f:
        for li, line in enumerate(f):
            vals = list(map(float, line.strip().split()))
            if len(vals) < 5 + 3:
                continue
            cls, xc, yc, bw, bh = vals[:5]
            kpts = vals[5:]
            pts = choose_projection(kpts, xc, yc, bw, bh, w, h)
            color = palette[li % len(palette)]
            img = draw_keypoints_with_ids(img, pts, color=color)

    # ★ 한 장씩 크게 표시
    plt.figure(figsize=(4, 3))  # 더 크게 보고 싶으면 (20, 15) 등으로 조정
    plt.imshow(img)
    plt.title(os.path.basename(img_path), fontsize=16)
    plt.axis("off")
    plt.tight_layout()
    plt.show()
